In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import io
import urllib.request


from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 네이버영화, 강사님 영화 리뷰 데이터 합친 파일 불러오기

In [51]:
#수집한 데이터, 강사님 데이터 합친 파일 불러오기
df = pd.read_csv('movie_ver1.csv', low_memory=False, encoding = 'UTF-8')
df.head()

,sentence,label
0,믿고보는 마블스토리 ㅋ,1
1,액션을 기대하고 봤다면 반대. 연기를 기대하고 봤다면 끝까지,0
2,사전정보 없이 봤는데 러닝타임 내내 시간 가는 줄 모르게 봤습니다,1
3,미쳤다.. 내 시간 순삭 당하고 옴,1
4,이 영화는 한마디로 헤리포터+제다이(스타워즈)+호빗+스타트렉 4가지의 영화를 합친...,1


In [52]:
print(df.isnull().sum())

sentence    0
label       0
dtype: int64


In [53]:
df_review = df['sentence']
df_review

0                                              믿고보는 마블스토리 ㅋ
1                         액션을 기대하고 봤다면 반대. 연기를 기대하고 봤다면 끝까지
2                      사전정보 없이 봤는데 러닝타임 내내 시간 가는 줄 모르게 봤습니다
3                                       미쳤다.. 내 시간 순삭 당하고 옴
4         이 영화는 한마디로 헤리포터+제다이(스타워즈)+호빗+스타트렉  4가지의 영화를 합친...
                                ...                        
209360            오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
209361         의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
209362                   그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다
209363       절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네
209364                                           마무리는 또 왜이래
Name: sentence, Length: 209365, dtype: object

## 전처리

In [54]:
#영어 소문자로 모두 변환
df_review = df_review.str.lower()

In [55]:
df_review[100:140]

100                                 지대 잘 만들었다 네이비실 위엄 지림
101                                    쿠키2개. 2편 존버탑니다...
102        간만에 조폭영화 재밌게 봤네...조폭영화 미화도 안하고 의리없는 게 딱 현실적임.
103                                        영화가 아닌 다큐멘터리 
104                                    포피파 피포파 포피파파피포파ㅠㅠ
105                     생각보다는 재밌었다. 주인공이나 빌런이나 서사 다 좋았음.
106                               빨리 d.c에서 제임스완 데려와요..ㅠㅠ
107                               진짜 꼭보길,, 역시 최민식 배우님 최고
108    분위기가 묘하게 독특하고 한국영화가 아닌 것 같은 느낌을 받았는데 감독이 프랑스인이...
109       배우연기가 아깝게 느껴지는 구멍 술술 설정과 어지러운 cg화면의 대환장 콜라보...
110    역시 믿고 보는 갓 연우진! 몰입도 높고 쫀쫀한 스토리 때문에 재미있게 봤습니다.....
111                                            사는게 지옥이다.
112    올가 배우를 이렇게 보게 되어서 너무 좋았어요! 앞으로 더 많은 작품 볼 수 있으면...
113    [스포있음] 방금 보고 나왔는데요…! 어… 엄청난 범죄영화 같이 홍보 해놓고 결말은...
114                       열일하시는 김동완 배우님에 눈물이 앞을 가립니다….ㅠㅠ
115                                     아... 이거뭔가요....ㅠㅠ
116                                     21세기에 맞게 뒤집어버리다!
117                     어색할 줄 알

In [56]:
#반복되는 문자 제거하기

df_review = df_review.str.replace("ㅋ","") # ㅋㅋㅋ제거
df_review = df_review.str.replace("ㅜ","") # ㅜㅜㅜ제거
df_review = df_review.str.replace("ㅠ","") # ㅠㅠㅠ제거
df_review = df_review.str.replace("ㅎ","") # ㅎㅎㅎ제거
df_review = df_review.str.replace("ㅇ","") # ㅇㅇ제거
df_review = df_review.str.replace("ㄷ","") # ㄷㄷ제거
df_review = df_review.str.replace("ㄱ","") # ㄱㄱ제거
df_review = df_review.str.replace("-","") # --제거
df_review = df_review.str.replace("_","") # __제거
df_review = df_review.str.replace("ㅡ","") # ㅡㅡ제거

#
df_review = df_review.str.replace("ㅉ","")
df_review = df_review.str.replace("ㅆㅈ","")
df_review = df_review.str.replace("ㅏ","")
df_review = df_review.str.replace("ㅡ","")

In [57]:
#단어 변환해주기
df_review = df_review.str.replace("ㄱㅊ","괜찮")

# 추가 단어 #
#줄임말 변경
df_review = df_review.str.replace("ㅈㄹ","지랄")
df_review = df_review.str.replace("ㅈㅁ","정말")
df_review = df_review.str.replace("ㅇㅈ","인정")
df_review = df_review.str.replace("ㄹㅇ","정말")
df_review = df_review.str.replace("ㅈㅅ","죄송")
df_review = df_review.str.replace("좋ㅈ었습니다","좋았습니다")
df_review = df_review.str.replace("ㅈㄴ","존나")


#영어표현 변경
df_review = df_review.str.replace("cg","씨지")
df_review = df_review.str.replace("ost","오에스티")
df_review = df_review.str.replace("sf","공상과학")
df_review = df_review.str.replace("tv","텔레비전")
df_review = df_review.str.replace("sns","소셜미디어")
df_review = df_review.str.replace("good","좋다")
df_review = df_review.str.replace("best","좋다")

In [58]:
#정규표현식 
df_review = df_review.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
df_review = df_review.str.replace('^ +', "") #공백은 empty값으로 변경
#df_review.drop_duplicates(subset = ['content'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
df_review.replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
df_review = df_review.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(df_review))

C:\Users\user\AppData\Local\Temp/ipykernel_1152/4275888759.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_review = df_review.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
C:\Users\user\AppData\Local\Temp/ipykernel_1152/4275888759.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_review = df_review.str.replace('^ +', "") #공백은 empty값으로 변경


전처리 후 테스트용 샘플의 개수 : 207611


In [59]:
df_review[:30]

0                                           믿고보는 마블스토리 
1                      액션을 기대하고 봤다면 반대 연기를 기대하고 봤다면 끝까지
2                  사전정보 없이 봤는데 러닝타임 내내 시간 가는 줄 모르게 봤습니다
3                                     미쳤다 내 시간 순삭 당하고 옴
4     이 영화는 한마디로 헤리포터제다이스타워즈호빗스타트렉  가지의 영화를 합친것 보다 배...
5                                       자레드 레토 연기 너무 잘함
6     느닷없는 개연성과 범죄스릴러에 억지로 로맨스 장르를 부합하는 어이없는 시도평소 영화...
7                   평점점이라서 꺼렸는데 누가 추천해줘서 봤는데 진짜 재미지게 봄 
8     베놈 보다는 훨씬 잘만들었다오에스티는 다크아니트 느낌이 강하게 느껴진다표절했나 이정...
9     마블 영화를 전체적으로 봐왔던 사람으로써 이 영화는 단순히 생각 없이 보길 추천함 ...
10                                         올해 최고의 코미디영화
11    경상도 토박인데 사투리 어색해서 집중 안됐음어색한 사투리웅얼웅얼 대사쳐서 뭔 말인지...
12    큰 기대를 안하고 관람해서 그런지아주 재미있지도 않고 아주 재미없지도 않았던 그런 ...
13                          생존하게 해준 고든이랑 샤샤만 살았어도 좋았을듯 
14                                         잔잔하니 감동적이네요 
15                                       진짜 영화가 미제다 미제 
16                                 진짜 재미없고 말 소리도 잘 안들리고
17                                         연기들은 

## py-hanspell 적용하기
naver 띄어쓰기, 맞춤법 검사기

In [60]:
df['sentence'] = df_review

In [61]:
df

,sentence,label
0,믿고보는 마블스토리,1
1,액션을 기대하고 봤다면 반대 연기를 기대하고 봤다면 끝까지,0
2,사전정보 없이 봤는데 러닝타임 내내 시간 가는 줄 모르게 봤습니다,1
3,미쳤다 내 시간 순삭 당하고 옴,1
4,이 영화는 한마디로 헤리포터제다이스타워즈호빗스타트렉 가지의 영화를 합친것 보다 배...,1
...,...,...
209360,오랜만에 평점 로긴했네 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
209361,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
209362,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
209363,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [75]:
print(df.isnull().sum())

sentence    1754
label          0
dtype: int64


In [76]:
df = df.dropna(how = 'any') # Null 값이 한개라도 존재하는 행 제거

In [77]:
print(df.isnull().sum())

sentence    0
label       0
dtype: int64


In [80]:
#hanspell은 500자까지만 분석해주기때문에
#500자 이상의 값은 넘어가게 함수 만들어줌

from hanspell import spell_checker

new_sent = []
def review_sent(text):
    for i in range(len(text)):
        spelled_sent = spell_checker.check(text[i])
        hanspell_sent = spelled_sent.checked
        sent = hanspell_sent
        new_sent.append(sent)
        

In [81]:
review_sent(df['sentence'])

KeyError: 686

In [47]:
df['sentence'] = new_sent

ValueError: Length of values (0) does not match length of index (209365)

In [52]:
df

,sentence,label
0,믿고보는 마블스토리,1
1,액션을 기대하고 봤다면 반대 연기를 기대하고 봤다면 끝까지,0
2,사전정보 없이 봤는데 러닝타임 내내 시간 가는 줄 모르게 봤습니다,1
3,미쳤다 내 시간 순삭 당하고 옴,1
4,이 영화는 한마디로 헤리포터제다이스타워즈호빗스타트렉 가지의 영화를 합친것 보다 배...,1
...,...,...
209360,오랜만에 평점 로긴했네 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
209361,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
209362,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
209363,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [53]:
#완성된 데이터파일 csv파일로 저장하기
df.to_csv('review_preprocessing_ver2.csv',index=False)

## 불용어 지정

In [33]:
#불용어지정하기 ->소연

#불용어지정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','요','이다','-']

#형태소분석기 정하기

## 형태소분석기_konlpy

In [83]:
from konlpy.tag import Kkma, Komoran, Okt

#mec = Mecab()
okt = Okt()
kkm = Kkma()
kom = Komoran()

In [84]:
import konlpy
print(konlpy.__version__)


0.6.0


In [ ]:
#io와 pandas 모듈 import
import io


from konlpy.tag import Kkma
from konlpy.utils import pprint